# Experiment 006: Grid-Based Translation Approach (egortrushin)

Implementing the grid-based translation approach that builds configurations from scratch.
This is fundamentally different from SA on existing configurations.

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import time

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def has_collision(trees):
    """Check for collisions using Shapely (ground truth)."""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [4]:
# Verify the repaired baseline
print("Verifying repaired baseline...")
df = pd.read_csv('/home/submission/submission.csv')

total_score = 0
overlap_count = 0
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        score = get_score(trees, n)
        total_score += score
        if has_collision(trees):
            overlap_count += 1
            if overlap_count <= 5:
                print(f"  N={n}: OVERLAP!")

print(f"\nTotal score: {total_score:.6f}")
print(f"Overlaps: {overlap_count}")
print(f"Gap to target: {total_score - 68.922808:.6f} points")

Verifying repaired baseline...



Total score: 70.682741
Overlaps: 0
Gap to target: 1.759933 points


In [5]:
# Grid-based translation approach from egortrushin kernel
# Creates trees in a grid pattern and optimizes the translation parameters

def create_grid_trees(n, dx, dy, angle1=0, angle2=180):
    """
    Create n trees in a grid pattern with alternating angles.
    dx, dy are the translation between adjacent trees.
    """
    trees = []
    
    # Find grid dimensions
    cols = int(math.ceil(math.sqrt(n * 1.5)))  # More columns than rows
    rows = int(math.ceil(n / cols))
    
    count = 0
    for row in range(rows):
        for col in range(cols):
            if count >= n:
                break
            
            # Alternating angles for interlocking
            angle = angle1 if (row + col) % 2 == 0 else angle2
            
            # Position with offset for odd rows
            x = col * dx
            if row % 2 == 1:
                x += dx / 2  # Offset for interlocking
            y = row * dy
            
            trees.append(ChristmasTree(str(x), str(y), str(angle)))
            count += 1
        if count >= n:
            break
    
    return trees

def optimize_grid_sa(n, n_steps=5000, T_max=1.0, T_min=0.001, seed=42):
    """
    Optimize grid parameters using simulated annealing.
    """
    random.seed(seed)
    
    # Initial parameters (based on tree dimensions)
    # Tree width is ~0.7, height is ~1.0
    best_dx = 0.7
    best_dy = 0.9
    best_angle1 = 0
    best_angle2 = 180
    
    trees = create_grid_trees(n, best_dx, best_dy, best_angle1, best_angle2)
    if has_collision(trees):
        # Increase spacing until no collision
        for scale in [1.1, 1.2, 1.3, 1.4, 1.5, 2.0]:
            trees = create_grid_trees(n, best_dx * scale, best_dy * scale, best_angle1, best_angle2)
            if not has_collision(trees):
                best_dx *= scale
                best_dy *= scale
                break
    
    best_side = get_side_length(trees)
    best_score = get_score(trees, n)
    
    # SA parameters
    alpha = (T_min / T_max) ** (1.0 / n_steps)
    T = T_max
    
    current_dx, current_dy = best_dx, best_dy
    current_angle1, current_angle2 = best_angle1, best_angle2
    current_side = best_side
    
    for step in range(n_steps):
        # Perturb parameters
        scale = T / T_max
        new_dx = current_dx + random.uniform(-0.05 * scale, 0.05 * scale)
        new_dy = current_dy + random.uniform(-0.05 * scale, 0.05 * scale)
        new_angle1 = (current_angle1 + random.uniform(-10 * scale, 10 * scale)) % 360
        new_angle2 = (current_angle2 + random.uniform(-10 * scale, 10 * scale)) % 360
        
        # Ensure positive spacing
        new_dx = max(0.3, new_dx)
        new_dy = max(0.3, new_dy)
        
        # Create new configuration
        trees = create_grid_trees(n, new_dx, new_dy, new_angle1, new_angle2)
        
        if not has_collision(trees):
            new_side = get_side_length(trees)
            delta = new_side - current_side
            
            if delta < 0 or random.random() < math.exp(-delta / T):
                current_dx, current_dy = new_dx, new_dy
                current_angle1, current_angle2 = new_angle1, new_angle2
                current_side = new_side
                
                if new_side < best_side:
                    best_dx, best_dy = new_dx, new_dy
                    best_angle1, best_angle2 = new_angle1, new_angle2
                    best_side = new_side
        
        T *= alpha
    
    # Return best configuration
    trees = create_grid_trees(n, best_dx, best_dy, best_angle1, best_angle2)
    return trees, best_side

print("Grid optimization functions defined")

Grid optimization functions defined


In [6]:
# Test grid approach on a few N values
print("Testing grid approach...")

test_ns = [10, 20, 50, 100]
for n in test_ns:
    baseline_trees = load_configuration_from_df(n, df)
    baseline_score = get_score(baseline_trees, n)
    
    start = time.time()
    grid_trees, grid_side = optimize_grid_sa(n, n_steps=3000, seed=42)
    elapsed = time.time() - start
    
    grid_score = get_score(grid_trees, n)
    has_overlap = has_collision(grid_trees)
    
    print(f"N={n}: baseline={baseline_score:.6f}, grid={grid_score:.6f}, overlap={has_overlap}, time={elapsed:.1f}s")

Testing grid approach...


N=10: baseline=0.376630, grid=1.088212, overlap=False, time=4.3s


N=20: baseline=0.376057, grid=0.989167, overlap=False, time=10.5s


N=50: baseline=0.360753, grid=0.843694, overlap=False, time=38.0s


N=100: baseline=0.345531, grid=0.837886, overlap=False, time=115.7s


In [7]:
# Backward propagation approach - start from N=200 and remove trees
# This is fundamentally different from local search

def backward_propagation(df, max_n=200):
    """
    Start from N=200 configuration and remove trees one by one.
    For each N from 199 down to 1, try removing each tree and keep the best.
    """
    print("Starting backward propagation...")
    
    # Load all configurations
    configs = {}
    scores = {}
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        configs[n] = trees
        scores[n] = get_score(trees, n)
    
    original_total = sum(scores.values())
    print(f"Original total score: {original_total:.6f}")
    
    improvements = 0
    
    # Start from N=200 and work down
    for n in range(max_n, 1, -1):
        target_n = n - 1
        current_best_score = scores[target_n]
        best_tree_to_remove = None
        best_new_trees = None
        
        # Try removing each tree from the N configuration
        source_trees = configs[n]
        
        for i in range(len(source_trees)):
            # Create configuration without tree i
            candidate_trees = [t.clone() for j, t in enumerate(source_trees) if j != i]
            
            # Check for collisions (shouldn't happen if we just remove a tree)
            if has_collision(candidate_trees):
                continue
            
            candidate_score = get_score(candidate_trees, target_n)
            
            if candidate_score < current_best_score:
                current_best_score = candidate_score
                best_tree_to_remove = i
                best_new_trees = candidate_trees
        
        # If we found an improvement, update
        if best_new_trees is not None:
            old_score = scores[target_n]
            configs[target_n] = best_new_trees
            scores[target_n] = current_best_score
            improvements += 1
            if n % 20 == 0 or improvements <= 5:
                print(f"  N={target_n}: improved {old_score:.6f} -> {current_best_score:.6f} (removed tree {best_tree_to_remove} from N={n})")
    
    new_total = sum(scores.values())
    print(f"\nBackward propagation complete!")
    print(f"Total improvements: {improvements}")
    print(f"New total score: {new_total:.6f}")
    print(f"Improvement: {original_total - new_total:.6f} points")
    
    return configs, scores

# Run backward propagation
bp_configs, bp_scores = backward_propagation(df)

Starting backward propagation...


Original total score: 70.682741


  N=137: improved 0.345264 -> 0.343528 (removed tree 118 from N=138)



Backward propagation complete!
Total improvements: 1
New total score: 70.681004
Improvement: 0.001737 points


In [8]:
# Analyze which N values have the most room for improvement
# by comparing their scores to theoretical lower bounds

print("Analyzing score distribution by N...")

# Calculate score per N
n_scores = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    n_scores[n] = get_score(trees, n)

# Find N values with highest scores (most room for improvement)
sorted_scores = sorted(n_scores.items(), key=lambda x: x[1], reverse=True)

print("\nTop 20 N values with highest scores (most room for improvement):")
for n, score in sorted_scores[:20]:
    print(f"  N={n}: score={score:.6f}")

print("\nBottom 20 N values with lowest scores (best optimized):")
for n, score in sorted_scores[-20:]:
    print(f"  N={n}: score={score:.6f}")

# Calculate cumulative contribution
cumulative = 0
print("\nCumulative score contribution:")
for i, (n, score) in enumerate(sorted_scores):
    cumulative += score
    if i < 10 or i % 20 == 0:
        print(f"  Top {i+1} N values contribute: {cumulative:.6f} ({100*cumulative/total_score:.1f}%)")

Analyzing score distribution by N...



Top 20 N values with highest scores (most room for improvement):
  N=1: score=0.661250
  N=2: score=0.450779
  N=3: score=0.434745
  N=5: score=0.416850
  N=4: score=0.416545
  N=7: score=0.399897
  N=6: score=0.399610
  N=9: score=0.387415
  N=8: score=0.385407
  N=15: score=0.379203
  N=10: score=0.376630
  N=21: score=0.376451
  N=20: score=0.376057
  N=11: score=0.375736
  N=22: score=0.375258
  N=16: score=0.374128
  N=26: score=0.373997
  N=12: score=0.372724
  N=13: score=0.372323
  N=25: score=0.372144

Bottom 20 N values with lowest scores (best optimized):
  N=153: score=0.336287
  N=197: score=0.336047
  N=165: score=0.335569
  N=192: score=0.335301
  N=166: score=0.334819
  N=178: score=0.334442
  N=154: score=0.334158
  N=193: score=0.333950
  N=132: score=0.333818
  N=196: score=0.333299
  N=194: score=0.333085
  N=195: score=0.332901
  N=167: score=0.332835
  N=179: score=0.332597
  N=168: score=0.332475
  N=155: score=0.332075
  N=180: score=0.331002
  N=182: score=0.3

In [9]:
# Focus on optimizing small N values with aggressive SA
# These contribute the most to the total score

def optimize_small_n_aggressive(n, trees, n_restarts=10, n_steps=10000, seed=42):
    """
    Aggressive SA optimization for small N values.
    Uses multiple restarts with different random seeds.
    """
    best_trees = [t.clone() for t in trees]
    best_score = get_score(best_trees, n)
    
    for restart in range(n_restarts):
        random.seed(seed + restart * 1000)
        
        # Start from current best
        current_trees = [t.clone() for t in best_trees]
        current_score = best_score
        
        # SA parameters
        T_max = 0.5
        T_min = 0.0001
        alpha = (T_min / T_max) ** (1.0 / n_steps)
        T = T_max
        
        for step in range(n_steps):
            # Pick a random tree
            i = random.randint(0, n - 1)
            
            # Save old state
            old_x = current_trees[i].center_x
            old_y = current_trees[i].center_y
            old_angle = current_trees[i].angle
            
            # Perturb
            scale = T / T_max
            dx = Decimal(str(random.uniform(-0.1 * scale, 0.1 * scale)))
            dy = Decimal(str(random.uniform(-0.1 * scale, 0.1 * scale)))
            dangle = Decimal(str(random.uniform(-30 * scale, 30 * scale)))
            
            new_x = old_x + dx
            new_y = old_y + dy
            new_angle = (old_angle + dangle) % Decimal('360')
            
            # Apply perturbation
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(new_angle))
            
            # Check collision
            if has_collision(current_trees):
                # Revert
                current_trees[i] = ChristmasTree(str(old_x), str(old_y), str(old_angle))
            else:
                new_score = get_score(current_trees, n)
                delta = new_score - current_score
                
                if delta < 0 or random.random() < math.exp(-delta / T):
                    current_score = new_score
                    if new_score < best_score:
                        best_score = new_score
                        best_trees = [t.clone() for t in current_trees]
                else:
                    # Revert
                    current_trees[i] = ChristmasTree(str(old_x), str(old_y), str(old_angle))
            
            T *= alpha
    
    return best_trees, best_score

# Optimize small N values
print("Optimizing small N values with aggressive SA...")

improved_configs = {}
improved_scores = {}
total_improvement = 0

for n in range(1, 21):  # Focus on N=1 to 20
    trees = load_configuration_from_df(n, df)
    original_score = get_score(trees, n)
    
    # More restarts for smaller N
    n_restarts = max(5, 20 - n)
    n_steps = max(5000, 20000 - n * 500)
    
    start = time.time()
    new_trees, new_score = optimize_small_n_aggressive(n, trees, n_restarts=n_restarts, n_steps=n_steps, seed=42+n)
    elapsed = time.time() - start
    
    improved_configs[n] = new_trees
    improved_scores[n] = new_score
    
    improvement = original_score - new_score
    total_improvement += improvement
    
    if improvement > 0:
        print(f"  N={n}: {original_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
    else:
        print(f"  N={n}: {original_score:.6f} (no improvement) [{elapsed:.1f}s]")

print(f"\nTotal improvement from small N optimization: {total_improvement:.6f} points")

Optimizing small N values with aggressive SA...


  N=1: 0.661250 (no improvement) [40.3s]


  N=2: 0.450779 (no improvement) [48.0s]


  N=3: 0.434745 (no improvement) [55.1s]


  N=4: 0.416545 (no improvement) [64.8s]


...

  N=16: 0.374128 (no improvement) [86.2s]


  N=17: 0.370040 (no improvement) [91.6s]


  N=18: 0.368771 (no improvement) [96.8s]


  N=19: 0.368615 (no improvement) [98.4s]


  N=20: 0.376057 (no improvement) [100.7s]

Total improvement from small N optimization: 0.000000 points


In [10]:
# Try ensemble approach - pick best configuration for each N from multiple CSVs
print("Trying ensemble approach from multiple CSVs...")

csv_files = [
    '/home/submission/submission.csv',  # Current best (repaired baseline)
    '/home/code/submission_candidates/candidate_000.csv',
    '/home/code/submission_candidates/candidate_001.csv',
    '/home/code/submission_candidates/candidate_002.csv',
    '/home/code/submission_candidates/candidate_003.csv',
    '/home/code/submission_candidates/candidate_004.csv',
]

# Load all CSVs
dfs = {}
for csv_file in csv_files:
    try:
        dfs[csv_file] = pd.read_csv(csv_file)
        print(f"Loaded {csv_file}")
    except Exception as e:
        print(f"Failed to load {csv_file}: {e}")

# For each N, find the best configuration across all CSVs
best_configs = {}
best_scores = {}
best_sources = {}

for n in range(1, 201):
    best_score = float('inf')
    best_trees = None
    best_source = None
    
    for csv_file, csv_df in dfs.items():
        try:
            trees = load_configuration_from_df(n, csv_df)
            if trees and not has_collision(trees):
                score = get_score(trees, n)
                if score < best_score:
                    best_score = score
                    best_trees = trees
                    best_source = csv_file
        except Exception as e:
            pass
    
    if best_trees:
        best_configs[n] = best_trees
        best_scores[n] = best_score
        best_sources[n] = best_source

# Calculate total score
ensemble_total = sum(best_scores.values())
print(f"\nEnsemble total score: {ensemble_total:.6f}")
print(f"Improvement over baseline: {total_score - ensemble_total:.6f} points")

# Show which CSVs contributed
source_counts = {}
for n, source in best_sources.items():
    source_counts[source] = source_counts.get(source, 0) + 1

print("\nSource contributions:")
for source, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {source}: {count} configurations")

Trying ensemble approach from multiple CSVs...
Loaded /home/submission/submission.csv
Loaded /home/code/submission_candidates/candidate_000.csv
Loaded /home/code/submission_candidates/candidate_001.csv
Loaded /home/code/submission_candidates/candidate_002.csv
Loaded /home/code/submission_candidates/candidate_003.csv
Loaded /home/code/submission_candidates/candidate_004.csv



Ensemble total score: 70.682741
Improvement over baseline: 0.000000 points

Source contributions:
  /home/submission/submission.csv: 200 configurations


In [11]:
# Implement swap moves - swap positions of two trees
# This can escape local optima that translation moves cannot

def swap_trees(trees, i, j):
    """Swap positions of trees i and j."""
    new_trees = [t.clone() for t in trees]
    
    # Swap positions (keep angles)
    x_i, y_i = new_trees[i].center_x, new_trees[i].center_y
    x_j, y_j = new_trees[j].center_x, new_trees[j].center_y
    
    new_trees[i] = ChristmasTree(str(x_j), str(y_j), str(new_trees[i].angle))
    new_trees[j] = ChristmasTree(str(x_i), str(y_i), str(new_trees[j].angle))
    
    return new_trees

def optimize_with_swaps(n, trees, n_iterations=1000, seed=42):
    """
    Optimize configuration using swap moves.
    """
    random.seed(seed)
    
    best_trees = [t.clone() for t in trees]
    best_score = get_score(best_trees, n)
    
    for iteration in range(n_iterations):
        # Pick two random trees
        i = random.randint(0, n - 1)
        j = random.randint(0, n - 1)
        if i == j:
            continue
        
        # Try swap
        new_trees = swap_trees(best_trees, i, j)
        
        if not has_collision(new_trees):
            new_score = get_score(new_trees, n)
            if new_score < best_score:
                best_score = new_score
                best_trees = new_trees
    
    return best_trees, best_score

# Test swap optimization on a few N values
print("Testing swap optimization...")

for n in [10, 20, 50, 100, 150, 200]:
    trees = load_configuration_from_df(n, df)
    original_score = get_score(trees, n)
    
    start = time.time()
    new_trees, new_score = optimize_with_swaps(n, trees, n_iterations=5000, seed=42+n)
    elapsed = time.time() - start
    
    improvement = original_score - new_score
    if improvement > 0:
        print(f"  N={n}: {original_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
    else:
        print(f"  N={n}: {original_score:.6f} (no improvement) [{elapsed:.1f}s]")

Testing swap optimization...


  N=10: 0.376630 (no improvement) [4.8s]


  N=20: 0.376057 (no improvement) [9.5s]


  N=50: 0.360753 (no improvement) [26.3s]


  N=100: 0.345531 (no improvement) [71.1s]


  N=150: 0.337065 (no improvement) [124.9s]


  N=200: 0.337731 (no improvement) [184.6s]


In [12]:
# Implement squeeze operation - scale all trees toward the center
# This can sometimes find improvements by compacting the configuration

def squeeze_trees(trees, scale_factor_val):
    """Scale all trees toward the center by scale_factor."""
    if not trees:
        return trees
    
    # Find center
    cx = sum(float(t.center_x) for t in trees) / len(trees)
    cy = sum(float(t.center_y) for t in trees) / len(trees)
    
    new_trees = []
    for t in trees:
        new_x = cx + (float(t.center_x) - cx) * scale_factor_val
        new_y = cy + (float(t.center_y) - cy) * scale_factor_val
        new_trees.append(ChristmasTree(str(new_x), str(new_y), str(t.angle)))
    
    return new_trees

def optimize_with_squeeze(n, trees, n_steps=100):
    """
    Try to squeeze the configuration to reduce bounding box.
    """
    best_trees = [t.clone() for t in trees]
    best_score = get_score(best_trees, n)
    
    # Try different squeeze factors
    for scale in np.linspace(0.999, 0.95, n_steps):
        new_trees = squeeze_trees(best_trees, scale)
        
        if not has_collision(new_trees):
            new_score = get_score(new_trees, n)
            if new_score < best_score:
                best_score = new_score
                best_trees = new_trees
        else:
            # Can't squeeze further without collision
            break
    
    return best_trees, best_score

# Test squeeze optimization
print("Testing squeeze optimization...")

for n in [10, 20, 50, 100, 150, 200]:
    trees = load_configuration_from_df(n, df)
    original_score = get_score(trees, n)
    
    start = time.time()
    new_trees, new_score = optimize_with_squeeze(n, trees, n_steps=200)
    elapsed = time.time() - start
    
    improvement = original_score - new_score
    if improvement > 0:
        print(f"  N={n}: {original_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
    else:
        print(f"  N={n}: {original_score:.6f} (no improvement) [{elapsed:.1f}s]")

Testing squeeze optimization...
  N=10: 0.376630 (no improvement) [0.0s]
  N=20: 0.376057 (no improvement) [0.0s]
  N=50: 0.360753 (no improvement) [0.0s]
  N=100: 0.345531 (no improvement) [0.0s]
  N=150: 0.337065 (no improvement) [0.0s]


  N=200: 0.337731 (no improvement) [0.1s]


In [ ]:
# Run grid optimization on all N values and compare with baseline
print("\nRunning grid optimization on all N values...")

best_configs = {}
best_scores = {}
improvements = 0

start_time = time.time()

for n in range(1, 201):
    baseline_trees = load_configuration_from_df(n, df)
    baseline_score = get_score(baseline_trees, n)
    
    # Adjust iterations based on N
    if n <= 20:
        n_steps = 5000
    elif n <= 50:
        n_steps = 3000
    else:
        n_steps = 2000
    
    grid_trees, grid_side = optimize_grid_sa(n, n_steps=n_steps, seed=42+n)
    grid_score = get_score(grid_trees, n)
    
    # Use grid if better and no overlap
    if grid_score < baseline_score and not has_collision(grid_trees):
        best_configs[n] = grid_trees
        best_scores[n] = grid_score
        improvements += 1
        if n % 20 == 0:
            print(f"N={n}: IMPROVED {baseline_score:.6f} -> {grid_score:.6f}")
    else:
        best_configs[n] = baseline_trees
        best_scores[n] = baseline_score
        if n % 20 == 0:
            print(f"N={n}: kept baseline {baseline_score:.6f} (grid={grid_score:.6f})")

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed:.1f}s")
print(f"Improvements: {improvements}")

new_total = sum(best_scores.values())
print(f"\nNew total score: {new_total:.6f}")
print(f"Improvement: {total_score - new_total:.6f} points")
print(f"Gap to target: {new_total - 68.922808:.6f} points")